In [1]:
from astropy.io import fits
from astropy.modeling import models
from astropy import units as u
from astropy import coordinates as coord
from asdf import AsdfFile

In [2]:
from gwcs import wcs
from gwcs import utils as gwutils
from gwcs import coordinate_frames as cf

In [3]:
# read distortion polynomials and filter offsets from reference files
distortion = AsdfFile.open('jwst_miri_distortion_0007.asdf').tree['model']
filter='F1130W'
filter_offset = AsdfFile.open('jwst_miri_filteroffset_0001.asdf').tree[filter]
filter_offset


{'column_offset': 0.15000001, 'row_offset': -0.58999997}

In [4]:
# combine to create the full distortion
full_distortion = models.Shift(filter_offset['row_offset']) & models.Shift(
        filter_offset['column_offset']) | distortion
full_distortion = full_distortion.rename('distortion')


In [17]:
#Create 3 coordinate frames for the WCS pipeline
detector = cf.Frame2D(name='detector', axes_order=(0,1), unit=(u.pix, u.pix))
focal = cf.Frame2D(name='focal', axes_order=(0,1), unit=(u.arcmin, u.arcmin))
sky = cf.CelestialFrame(name='icrs', reference_frame=coord.ICRS())


In [18]:
# create the FITS Standard transform, reaing keywords from the header
f = fits.open('miri_image.fits')
header = f[0].header

fits_transform = gwutils.make_fitswcs_transform(header)


In [19]:
pipeline = [(detector, distortion),
                (focal, fits_transform),
                (sky, None)
                ]
w = wcs.WCS(pipeline)

In [20]:
w(20, 25)

(242.8483213442464, -46.841218283308145)

In [21]:
w.backward_transform(*w(100, 65))

(98.73748221335109, 66.84884083321418)

In [22]:
fwcs = AsdfFile()
fwcs.tree['wcs'] = w
fwcs.write_to('miri_wcs.asdf')

In [23]:
w.available_frames

['detector', 'focal', 'icrs']

In [24]:
w.icrs

<CelestialFrame(name="icrs", unit=(Unit("deg"), Unit("deg")), axes_names=('lon', 'lat'), axes_order=(0, 1), reference_frame=<ICRS Frame>)>

In [30]:
w.icrs.transform_to('galactic', 100, 60)

<SkyCoord (Galactic): (l, b) in deg
    (334.5871473, 3.37618984)>